In [1]:
import numpy as np
import math
import ipywidgets as widgets
from ipycanvas import Canvas, hold_canvas
from time import sleep

# DEM 3D 可视化

## 常用矩阵变换函数

In [2]:
def rotate_x(vertices, a):
    m = np.array([
        [1, 0, 0, 0],
        [0, math.cos(a), math.sin(a), 0],
        [0, -math.sin(a), math.cos(a), 0],
        [0, 0, 0, 1]
    ])
    return vertices @ m

def rotate_y(vertices, a):
    m = np.array([
        [math.cos(a), 0, -math.sin(a), 0],
        [0, 1, 0, 0],
        [math.sin(a), 0, math.cos(a), 0],
        [0, 0, 0, 1]
    ])
    return vertices @ m

def rotate_z(vertices, a):
    m = np.array([
        [math.cos(a), math.sin(a), 0, 0],
        [-math.sin(a), math.cos(a), 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])
    return vertices @ m

def translate(vertices, dx, dy, dz):
    m = np.array([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [dx, dy, dz, 1],
    ])
    return vertices @ m

def scale(vertices, sx, sy, sz):
    m = np.array([
        [sx, 0, 0, 0],
        [0, sy, 0, 0],
        [0, 0, sz, 0],
        [0, 0, 0, 1]
    ])
    return vertices @ m

## 3D可视化场景对象

In [3]:
class Object3D():
    def __init__(self, vertices, facets):
        self.vertices = vertices
        self.facets = facets
    
    def move(self, dx, dy, dz):
        self.vertices = self.vertices @ translate(dx, dy, dz)
    def rotate_x(self, d):
        self.vertices = self.vertices @ rotate_x(d)
    def rotate_y(self, d):
        self.vertices = self.vertices @ rotate_y(d)
    def rotate_z(self, d):
        self.vertices = self.vertices @ rotate_z(d)

class Camera:
    def __init__(self, position, heading, pitch, yaw):
        self.position = position
        self.heading = heading
        self.pitch = pitch
        self.yaw = yaw
    def rotate_matrix(self):
        return rotate_z(rotate_x(rotate_y(np.eye(4), -self.heading), -self.pitch), -self.yaw)
    def translate_matrix(self):
        return translate(np.eye(4), -self.position[0], -self.position[1], -self.position[2])
    def view_matrix(self):
        return self.translate_matrix() @ self.rotate_matrix()
    
class Render():
    def __init__(self, height, width, camera:Camera):
        self.height = height
        self.width = width
        self.canvas = Canvas(height=self.height, width=self.width)
        self.objects = []
        self.camera = camera
        self.projection_matrix, self.to_screen_matrix = self.projection()
        self.show_axis = False
    
    def projection(self):
        h_fov = math.pi / 3
        v_fov = h_fov * (self.height / self.width)

        NEAR = 0.1
        FAR = 100
        RIGHT = math.tan(h_fov / 2)
        LEFT = -RIGHT
        TOP = math.tan(v_fov / 2)
        BOTTOM = -TOP

        m00 = 2.0 / (RIGHT - LEFT)
        m11 = 2.0 / (TOP - BOTTOM)
        m22 = (FAR + NEAR)*1.0 / (FAR - NEAR)
        m32 = -2.0 * NEAR * FAR / (FAR - NEAR)

        # 投影矩阵
        projection_matrix = np.array([
            [m00, 0, 0, 0],
            [0, m11, 0, 0],
            [0, 0, m22, 1],
            [0, 0, m32, 0]
        ])

        # 视口变换矩阵
        HW, HH = self.width/2, self.height/2
        to_screen_matrix = np.array([
            [HW, 0, 0, 0],
            [0, -HH, 0, 0],
            [0, 0, 1, 0],
            [HW, HH, 0, 1]
        ])
        return projection_matrix, to_screen_matrix
    
    def add_object(self, obj:Object3D):
        self.objects.append(obj)
    
    @staticmethod
    def _draw_seg(canvas:Canvas, v1, v2):
        if any(np.isnan(v1)) or any(np.isnan(v2)):
            return
        canvas.stroke_line(
            int(v1[0]), 
            int(v1[1]), 
            int(v2[0]), 
            int(v2[1]))
        
    def _draw_obj(self, obj:Object3D, clip=True):
        
        vertices = obj.vertices
        vertices[:,2] = vertices[:,2] * -1 # z值正负转换
        vertices = vertices @ self.camera.view_matrix() # 相机观察V
        vertices = vertices @ self.projection_matrix # 投影P
        vertices /= vertices[:, -1].reshape(-1, 1) # W值归一化
        
        # 裁剪
        if clip:
            vertices[(vertices > 2) | (vertices < -2)] = None
        
        vertices = vertices @ self.to_screen_matrix # 视口变换
        vertices = vertices[:,:2] # 取变换后的二维坐标
        
        # 屏幕绘制
        for facet in obj.facets:
            for i in range(1, len(facet)):
                Render._draw_seg(self.canvas, vertices[facet[i-1]], vertices[facet[i]])
    
    def _draw_axis(self):
        xaxis_obj = Object3D(vertices=np.array([
            [-100, 0, 0, 1],
            [-50, 0, 0, 1],
            [0, 0, 0, 1],
            [50, 0, 0, 1],
            [100, 0, 0, 1],
        ]), facets=np.array([
            [0,1,2,3,4]
        ]))
        yaxis_obj = Object3D(vertices=np.array([
            [0, -100, 0, 1],
            [0, 50, 0, 1],
            [0, 0, 0, 1],
            [0, 50, 0, 1],
            [0, 100, 0, 1],
        ]), facets=np.array([
            [0,1,2,3,4]
        ]))
        zaxis_obj = Object3D(vertices=np.array([
            [0, 0, -100, 1],
            [0, 0, -50, 1],
            [0, 0, 0, 1],
            [0, 0, 50, 1],
            [0, 0, 100, 1],
        ]), facets=np.array([
            [0,1,2,3,4]
        ]))
        old_stroke_style = self.canvas.stroke_style
        old_line_width = self.canvas.line_width
        self.canvas.line_width = 1
        self.canvas.stroke_style = 'red'
        self._draw_obj(xaxis_obj, clip=False)
        self.canvas.stroke_style = 'green'
        self._draw_obj(yaxis_obj, clip=False)
        self.canvas.stroke_style = 'blue'
        self._draw_obj(zaxis_obj, clip=False)
        self.canvas.stroke_style = old_stroke_style
        self.canvas.line_width = old_line_width
    
    def _render(self):
        with hold_canvas():
            if self.show_axis:
                self._draw_axis()
            for obj in self.objects:
                self._draw_obj(obj)
    
    def show(self):
        display(self.canvas)
        self._render()
        
    
    def refresh(self):
        self.canvas.clear()
        self._render()

## 创建场景

In [4]:
camera = Camera([117.4,29.81,-1.8], math.radians(0),math.radians(-27.8),math.radians(0))
render = Render(height=768, width=1024, camera=camera)
render.show_axis = True

## 添加对象
将DEM转换为MESH

In [5]:
import rasterio
from rasterio.enums import Resampling

# 重采样系数
upscale_factor = 1/40

# 高度拉伸系数
z_factor = 111120/50

with rasterio.open("../data/n30_e117_1arc_v3.dt2") as dataset:
    bound = dataset.bounds
    
    data = dataset.read(
        out_shape=(
            dataset.count,
            int(dataset.height * upscale_factor),
            int(dataset.width * upscale_factor)
        ),
        resampling=Resampling.bilinear
    )[0]
    
    data = data.astype('float64')
    data /= z_factor
    h,w = data.shape
    xres = (bound.right-bound.left)/w
    yres = (bound.top-bound.bottom)/h
    
    
    widx, hidx = np.meshgrid(range(w), range(h))
    Y = bound.top - hidx * yres
    X = bound.left + widx * xres
    
    vertices = np.hstack((
        X.reshape((-1,1)), 
        Y.reshape((-1,1)), 
        data.reshape((-1,1)), 
        np.ones(shape=(h,w)).reshape((-1,1))
    ))
    
    facets = []
    for i in range(1, h):
        for j in range(1, w):
            facets.append([(i-1)*w + (j-1), (i)*w + (j-1), (i)*w + (j), (i-1)*w + (j), (i-1)*w + (j-1)])

obj1 = Object3D(vertices=vertices, facets=np.array(facets))
render.add_object(obj1)

## 可视化

In [6]:
render.show()

headingSlider = widgets.FloatSlider(value=math.degrees(render.camera.heading), min=-180, max=180, step=0.01, description='heading:', layout=widgets.Layout(width='1000px'))
def headingChanged(evt):
    render.camera.heading = math.radians(evt.new)
    render.refresh()
headingSlider.observe(headingChanged, 'value')

pitchSlider = widgets.FloatSlider(value=math.degrees(render.camera.pitch), min=-180, max=180, step=0.01, description='pitch:', layout=widgets.Layout(width='1000px'))
def pitchChanged(evt):
    render.camera.pitch = math.radians(evt.new)
    render.refresh()
pitchSlider.observe(pitchChanged, 'value')

yawSlider = widgets.FloatSlider(value=math.degrees(render.camera.yaw), min=-180, max=180, step=0.01, description='yaw:', layout=widgets.Layout(width='1000px'))
def yewChanged(evt):
    render.camera.yaw = math.radians(evt.new)
    render.refresh()
yawSlider.observe(yewChanged, 'value')

xSlider = widgets.FloatSlider(value=render.camera.position[0], min=116, max=118, step=0.01, description='x:', layout=widgets.Layout(width='1000px'))
def xChanged(evt):
    render.camera.position[0] = evt.new
    render.refresh()
xSlider.observe(xChanged, 'value')

ySlider = widgets.FloatSlider(value=render.camera.position[1], min=28, max=32, step=0.01, description='y:', layout=widgets.Layout(width='1000px'))
def yChanged(evt):
    render.camera.position[1] = evt.new
    render.refresh()
ySlider.observe(yChanged, 'value')

zSlider = widgets.FloatSlider(value=render.camera.position[2], min=-10, max=10, step=0.01, description='z:', layout=widgets.Layout(width='1000px'))
def zChanged(evt):
    render.camera.position[2] = evt.new
    render.refresh()
zSlider.observe(zChanged, 'value')

widgets.VBox([headingSlider, pitchSlider, yawSlider, xSlider, ySlider, zSlider])

Canvas(height=768, width=1024)